In [30]:
using SymPy
import LinearAlgebra: norm, dot, qr, I

Functions for Power Method :)

In [75]:
function leadEigen(A) #computes leading (eigenvector, eigenvalue) pair
    #A is nxn
    n      = size(A)[2] #number of columns 
    u_0    = rand(n) #some random start vector
    u_1    = A*u_0
    u_prev = u_0 
    u_curr = u_1/norm(u_1) #normalize
    lambda = 0
    i = 0
    while(u_curr!=u_prev && i<=1000)#max 1000 iterations if res must b approx
        u_prev  = u_curr
        u_curr  = A*u_prev #repeatedly multiply by A (power method)
        u_curr /= norm(u_curr)
        i+=1
    end
    lambda  = (u_curr')*A*u_curr
    return (u_curr, lambda)
end
A = [2 1 6; 2 3 5; 8 9 0]
B = [4 3 2 5;
     9 8 8 1;
     9 3 9 2;
     6 0 4 0]
leadEigen(B)

([0.30798633001944875, 0.7260722934130327, 0.5723504095009087, 0.22445145132797034], 18.43302680219626)

In [32]:
function leadLeftSing(A) #computes leading left singular (vector, value) pair
    #A is mxn
    V     = (A')*A
    (v, σ) = leadEigen(V)
    sigma = σ^0.5 #singular vals roots of evals
    return (v, sigma)
end
A = [2 1 6; 2 3 7]
leadLeftSing(A)

([0.2790282328184172, 0.2938084077581624, 0.9142318441297286], 10.080296311193793)

In [33]:
function leadRightSing(A) #computes leading right singular (vector, value) pair
    (v, σ) = leadLeftSing(A)
    u = A*v/σ
    return u
end
A = [2 1 6; 2 3 7]
leadRightSing(A)

2-element Vector{Float64}:
 0.6286775450376474
 0.7776660879615599

In [74]:
function ithEigen(A, i) #computes ith (eigenvector, eigenvalue) pair
    j = 1
    (v, λ) = leadEigen(A) 
    while(j < i)
        A = A - λ*v*(v') #deflation to compute ith eval
        (v, λ) = leadEigen(A) 
        j += 1
    end
    return (v, λ)
end
A = [2 1 6; 2 3 5; 8 9 0]
B = [4 3 2 5;
     9 8 8 1;
     9 3 9 2;
     6 0 4 0]

ithEigen(B, 3)


([0.0983104466273777, 0.44254472923772786, 0.7177803926621973, 0.5284699864869654], -3.705916043489471)

In [35]:
function ithLeftSing(A, i) #computes ith left singular (vector, value) pair
    V = (A')*A
    (v, σ) = ithEigen(V, i) 
    sigma = σ^0.5 #singular vals roots of eigen vals
    return (v, sigma)
end
A = [2 1 6; 2 3 7]
ithLeftSing(A, 2)

([-0.25295694677442165, 0.9409079791373699, -0.22517761406097403], 1.1779754999713552)

In [36]:
function ithRightSing(A, i) #computes ith right singular (vector, value) pair
    (v, σ) = ithLeftSing(A, i) 
    u = A*v/σ #proven in class
    return u
end
A = [2 1 6; 2 3 7]
ithRightSing(A, 2)

2-element Vector{Float64}:
 -0.7776660879615598
  0.6286775450376471

Functions for QR Method

In [39]:
function isUpperTri(A) #returns true is matrix is (approx) upper tri
    n = size(A)[1] 
    for i = 1:n
        for j = 1:n
            if(i > j && abs(A[i, j]) >= 1*10^-100) #approximating 0s subdiag
                return false
            end
        end
    end
    return true
end

isUpperTri (generic function with 1 method)

In [60]:
function QREVal(A) #returns leading eigenvalue using QR method
    if(isUpperTri(A)) #returns when A converges to upper tri matrix
        return A[1,1] #eigenvals on diagonal (leading eval at top left)
    end
    (Q, R) = qr(A) #general qr method
    B = R*Q
    return QREVal(B)
end
A = [2 1 6; 
     2 3 5; 
     8 9 0]
B = [4 3 2 5;
     9 8 8 1;
     9 3 9 2;
     6 0 4 0]
B = [2 0 2 0 2;
     0 3 0 3 0;
     2 0 2 0 2; 
     0 3 0 3 0; 
     2 0 2 0 2]
QREVal(B)

5.999999999999999

In [61]:
function QREValShift(A) #use shifting to increase speed
    n = size(A)[2]
    if(isUpperTri(A))
        return A[1,1]
    end
    (Q, R) = qr(A)
    q = Q[:,n] #last column of Q
    shift = (q')*(A*q)/((q')*q) #raleigh quotient shift
    (Q, R) = qr(A - shift*I)
    B = R*Q + shift*I
    return QREValShift(B)
end
C = [4 3 2 5;
     9 8 8 1;
     9 3 9 2;
     6 0 4 0]
B = [2 0 2 0 2;
     0 3 0 3 0;
     2 0 2 0 2; 
     0 3 0 3 0; 
     2 0 2 0 2]
QREValShift(B)

5.999999999999999

In [44]:
function QRithEVal(A, i) #returns ith eval
    if(isUpperTri(A))
        return A[i,i] #ith eval is ith elem on diagonal
    end
    (Q, R) = qr(A) #general qr method
    B = R*Q
    return QRithEVal(B, i)
end
B = [4 3 2 5;
     9 8 8 1;
     9 3 9 2;
     6 0 4 0]
B = [2 0 2 0 2;
     0 3 0 3 0;
     2 0 2 0 2; 
     0 3 0 3 0; 
     2 0 2 0 2]
QRithEVal(B, 3)   

0.0

In [45]:
function QREValMat(A) #returns resultant upper tri matrix A w/ evals on diag
    if(isUpperTri(A)) #only returns when A is upper tri
        return A
    end
    (Q, R) = qr(A) #general qr method
    B = R*Q
    return QREValMat(B)
end
A = [2 1 6; 2 3 5; 8 9 0]
B = [4 3 2 5 3;
     9 8 8 1 1;
     9 3 9 2 4;
     6 0 4 0 1;
     5 9 2 6 5]
B = [2 0 2 0 2;
     0 3 0 3 0;
     2 0 2 0 2; 
     0 3 0 3 0; 
     2 0 2 0 2]
QREValMat(B)

5×5 Matrix{Float64}:
 6.0  0.0  0.0   0.0          0.0
 0.0  6.0  0.0  -8.88178e-16  0.0
 0.0  0.0  0.0   0.0          0.0
 0.0  0.0  0.0   0.0          0.0
 0.0  0.0  0.0   0.0          0.0

In [46]:
function QRDet(A)
    det = 1 
    B = QREValMat(A) #computes upper triangular matrix using QR method
    n = size(A)[1]
    for i = 1:n 
     lambda = B[i,i]
     if lambda == 0 #returns 0 immediately once we get 0 eigenvalue
        return 0
     end
     det *= lambda #determinant is produt of eigenvalues
    end
    return det
end
A = [2 1 6; 2 3 5; 8 9 0]
B = [4 3 2 5;
     9 8 8 1;
     9 3 9 2;
     6 0 4 0]
QRDet(B)

-609.9999999999999

In [48]:
function QRLSingVal(A) #computes leading left singular value
    V = (A')*A
    lambda = QREVal(V)
    sigma = lambda^0.5
    return sigma
end
A = [2 1 6; 2 3 7]
QRLSingVal(A)

10.080296311193795

In [58]:
function QRRSingVal(A) #computes leading right singular value
    U = A*A'
    lambda = QREVal(U)
    sigma = lambda^0.5
    return sigma
end
A = [3 1 4; 1 5 9; 2 6 5; 3 5 8]
QRRSingVal(A)

16.7868829209782

In [50]:
function QAcc(A, Q) #accumulator function to multiply all computed Q's
    if(isUpperTri(A))
        return Q
    end
    (Q_curr, R) = qr(A)
    B = R*Q_curr
    return QAcc(B, Q*Q_curr)
end

QAcc (generic function with 1 method)

In [82]:
function QRLeadEVec(A) #returns leading eigenvector of matrix A
    (Q, R) = qr(A)
    B = R*Q
    X = QAcc(B, Q) #A = XT(X') is the Schur decomposition
    X /= norm(X)   #cols of X are Schur vectors, first Schur vec = leading evec
    return X[:,1]
end
A = [2 1 6; 2 3 5; 8 9 0]
B = [4 3 2 5;
     9 8 8 1;
     9 3 9 2;
     6 0 4 0]
lambda = QREVal(B)
QRLeadEVec(A)

3-element Vector{Float64}:
 0.2792467837311729
 0.2941869101050574
 0.4108632729168768

In [57]:
function QRLSingVec(A) #leading left singular vector is leading evec of V
    V = (A')*A
    v = QRLeadEVec(V)[:,1]
    v /= norm(v)
    return v
end
A = [3 1 4; 1 5 9; 2 6 5; 3 5 8]
QRLSingVec(A)

3-element Vector{Float64}:
 -0.24422363818800713
 -0.5371016198697047
 -0.8073887938803415

In [59]:
function QRRSingVec(A) #leading left singular vector is leading evec of U
    v = QRLSingVec(A)
    sigma = QRLSingVal(A)
    u = A*v/sigma
    u /= norm(u)
    return u
end
A = [3 1 4; 1 5 9; 2 6 5; 3 5 8]
QRRSingVec(A)

4-element Vector{Float64}:
 -0.26802639484262936
 -0.6073927441119876
 -0.46155090265825616
 -0.5883932956136751

In [ ]:
function getR(A) #extra function not used, included to show effort haha
    Q = gramSchmidt(A)
    n = size(A)[2]
    R = zeros(n)
    for i = 1:(n-1)
        R = hcat(R, zeros(n))
    end
    for i = 1:n
        q = Q[:,i]
        for j = 1:n
            a = A[:,j]
            R[i, j] = (q')*a
            if(abs(R[i, j]) < 1*10^(-10))
                R[i, j] = 0
            end
        end
    end
    return R
end
A = [2 1 6; 2 3 5; 8 9 0]
getR(A)
    

3×3 Matrix{Float64}:
 8.48528  9.42809   2.59272
 0.0      1.45297  -2.37063
 0.0      0.0       6.97552